# Simulation Experiments of Braitenberg Vehicles using Genetic Algorithms

このノートブックでは、Braitenberg 車両を用いて、Spatial GA と Full Microbial GA の2種類の遺伝的アルゴリズムの性能比較実験を行います。  
実験はシミュレーション環境上で行い、各 GA の実験結果として車両の軌跡とフィットネス推移をグラフとして可視化します。



In [1]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import torch  # 必要ならば
import copy

# ノートブック用に出力設定を行う (例: %matplotlib inline)
%matplotlib inline

# src ディレクトリのコードをインポートする例
from braitenberg import Braitenberg
from environment import Environment
from ga_algorithms import SpatialGA, FullMicrobialGA
from utils import plot_trajectories, plot_fitness_trends, save_results, run_ga_algorithm

# ※上記のインポート文は、srcフォルダ内の各モジュールに合わせて適宜修正してください。

# パラメータ定義
starting_position = (3, 3)
starting_bearing = 10
runtime = 5
light_source = (0, 0)
noise_levels = [0.05]  # 例としてノイズレベルを1種類指定
generations_list = [50, 150, 300]
encoding_method = "direct"

# エンコーディングクラスの生成（DirectEncodingの場合）
# from src.encoding import DirectEncoding
# encoder = DirectEncoding()


ModuleNotFoundError: No module named 'braitenberg'

## 実験手順

以下の実験では、各ノイズレベルおよび世代数の条件下で Spatial GA と Full Microbial GA を実行し、
- 各アルゴリズムの軌跡 (trajectory)
- フィットネス推移 (fitness trend)
- 最適ゲノタイプ

を取得・可視化します。


In [ ]:
# 結果表示用のリストを初期化
all_trajectories = []
all_fitness_trends = []

# ノイズレベルに応じた実験実行
for noise_stddev in noise_levels:
    env = Environment(noise_stddev=noise_stddev)
    for generations in generations_list:
        trajectories = []
        fitness_trends = []

        # --- Spatial GA の実行 ---
        spatial_ga = SpatialGA(population_size=10, generations=generations, grid_size=(4, 4))
        trajectory_spatial, best_fitness_spatial, fitness_trend_spatial, best_genotype_spatial = run_ga_algorithm(
            spatial_ga, utils=None, encoder=encoder,
            starting_position=starting_position, starting_bearing=starting_bearing,
            env=env, runtime=runtime, noise_stddev=noise_stddev, encoding_method=encoding_method
        )
        trajectories.append((trajectory_spatial, f'Spatial GA ({generations} generations)'))
        fitness_trends.append((fitness_trend_spatial, f'Spatial GA ({generations} generations)'))
        save_results(f'results_spatial_ga_{generations}_noise_{noise_stddev}.txt', best_genotype_spatial, best_fitness_spatial)

        # --- Full Microbial GA の実行 ---
        full_microbial_ga = FullMicrobialGA(generations=generations)
        trajectory_full_microbial, best_fitness_full_microbial, fitness_trend_full_microbial, best_genotype_full_microbial = run_ga_algorithm(
            full_microbial_ga, utils=None, encoder=encoder,
            starting_position=starting_position, starting_bearing=starting_bearing,
            env=env, runtime=runtime, noise_stddev=noise_stddev, encoding_method=encoding_method
        )
        trajectories.append((trajectory_full_microbial, f'Full Microbial GA ({generations} generations)'))
        fitness_trends.append((fitness_trend_full_microbial, f'Full Microbial GA ({generations} generations)'))
        save_results(f'results_full_microbial_ga_{generations}_noise_{noise_stddev}.txt', best_genotype_full_microbial, best_fitness_full_microbial)

        # プロットの実施
        plot_trajectories(trajectories, light_source, f'Trajectories ({generations} generations, noise {noise_stddev})')
        plot_fitness_trends(fitness_trends, f'Fitness Trends ({generations} generations, noise {noise_stddev})')

        # 全結果をリストに追加（後でまとめて分析する場合に活用）
        all_trajectories.append((generations, trajectories))
        all_fitness_trends.append((generations, fitness_trends))


## 考察

各世代条件下でのGA実験結果から、下記の点が確認されました：
- Spatial GA と Full Microbial GA の収束速度や最適ゲノタイプに違いが見られる。
- ノイズレベルや初期条件の影響について、シミュレーション結果から評価可能。

これらの結果を元に、次のノートブック「Experiment_Results.ipynb」で、より詳細な結果の統計解析と比較を行います。
